Copyright © 2023 Gurobi Optimization, LLC

# 使用线性规划计算文本差异度

在本案例中,我们将通过一个独特的示例来演示如何应用优化技术来评估文本的差异性。这种技术有很多潜在的应用,比如检测抄袭、信息检索、聚类分析、文本分类、主题检测、问答系统、机器翻译和文本摘要等。更多信息请参阅[这里](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=1aff7b429f99f529228a4299a5794971adeb1ca3#:~:text=There%20are%20several%20applications%20or,machine%20translation%2C%20text%20summarization%20etc)。

**词移距离(Word Mover's Distance, WMD)** 是一种流行的文本相似度度量方法,用于衡量两份文档之间的语义距离。在本案例中,我们将实现两个目标:
- 给定两段文本,将WMD建模为优化问题并计算它
- 检查一本书中被抄袭的段落,然后在该书中找到与给定段落语义最接近的原始段落

|<img src="https://raw.githubusercontent.com/Gurobi/modeling-examples/master/text_dissimilarity/figure_obama.png" width="500" align="center">| 
|:--:|
|用词移距离来衡量两份文档的相似度示意图。<b>图片来源: [Towards AI](https://towardsai.net/p/nlp/word-movers-distance-wmd-explained-an-effective-method-of-document-classification-89cb258401f4) </b>| 

## I. 数据



### Google词向量数据

为了找到词语之间的语义距离,我们首先需要为每个词构建向量嵌入。
我们使用来自Google News的流行'word2vec'数据集。该数据集包含了300万个预训练的词向量嵌入。点击[这里](https://code.google.com/archive/p/word2vec/)了解更多关于该数据集的信息。

现在我们导入所有需要的包并下载word2vec数据。注意,在下面的代码中,下载数据大约需要一分钟。

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt   
from gensim.models import KeyedVectors 
from scipy import spatial

import gensim.downloader as api
model = api.load('word2vec-google-news-300')


我们已经下载了大量的词向量嵌入数据集。例如,"Sherlock"这个词在300维空间中的嵌入向量如下所示。

In [ ]:
model['Sherlock']

### 输入两份文档

接下来,我们创建要比较的两份文档。在这个例子中,我们看一下来自阿瑟·柯南·道尔的《福尔摩斯探案集》(最早的数据驱动型侦探小说)中的一句话:

"The little man stood glancing from one to the other of us with half-frightened, half-hopeful eyes, as one who is not sure whether he is on the verge of a windfall or of a catastrophe."


我们使用流行的文本生成技术([ChatGPT](https://chat.openai.com/chat))来构建一个语义相似的句子。

<img src="https://raw.githubusercontent.com/Gurobi/modeling-examples/master/text_dissimilarity/chatgpt clip.gif" width="750" align="center">


我们将这两个句子(文档)存储为字符串。你也可以尝试其他句子对。一些示例在下面的注释中给出。

In [ ]:
document1 = 'The little man stood glancing from one to the other of us with half-frightened, half-hopeful eyes, as one who is not sure whether he is on the verge of a windfall or of a catastrophe.'
document2 = 'With a gaze that shifted back and forth between us, the diminutive figure appeared to be a mixture of apprehension and anticipation, uncertain if he was on the cusp of a fortune or a disaster.'

# document1 = 'I barely saw Sherlock recently.'
# document2 = 'Lately, I have had little opportunity to catch a glimpse of Sherlock.'

# document1 = 'Obama speaks to the media in Illinois.'
# document2 = 'The President greets the press in Chicago'

## II. 文本预处理

在比较两个文档之前,我们首先要删除那些可能不携带任何语义价值的词,例如介词(如in、on、under)、连词(如and、for、but)和限定词(如a、an、the、another)。这些词被称为**停用词**。更多信息请参见[这里](https://kavita-ganesan.com/what-are-stop-words/#.ZCCurezMLt0)。
我们还要删除标点符号和专有名词,然后将所有单词转换为小写形式。

我们使用nltk包来实现这一点。在下面的代码中,包含"nltk.download"的行需要在第一次运行这个notebook时执行。后续运行时可以注释掉这些行。

In [ ]:
import nltk 
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')  
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import PorterStemmer 
 

将每个文档拆分成单词(或"标记")。每个单词被分类到几个类别中。例如,'NN'对应单数名词,'VBD'是过去时态动词。完整的分类列表请参见[这里](https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/)。

In [5]:
tagged_doc1 = nltk.tag.pos_tag(document1.split())
tagged_doc2 = nltk.tag.pos_tag(document2.split()) 
print(tagged_doc1)
print(tagged_doc2) 
 

了解了这些类别后,我们首先移除专有名词,即分类为NNP(单数专有名词)和NNPS(复数专有名词)的词。

In [6]:
edited_sentence1 = [word for word,tag in tagged_doc1 if tag not in ['NNP','NNPS']]
edited_sentence2 = [word for word,tag in tagged_doc2 if tag not in ['NNP','NNPS']] 
print(edited_sentence1)
print(edited_sentence2) 
 

接下来,我们删除所有的标点符号。

In [7]:
tokenizer = RegexpTokenizer(r'\w+') 
processed_doc1 = tokenizer.tokenize(' '.join(edited_sentence1))
processed_doc2 = tokenizer.tokenize(' '.join(edited_sentence2))
print(processed_doc1)
print(processed_doc2) 

现在我们删除所有的停用词。

In [8]:
processed_doc1 = [x.lower() for x in processed_doc1]
processed_doc2 = [x.lower() for x in processed_doc2]
processed_doc1 = [i for i in processed_doc1 if i not in stopwords.words('english')]
processed_doc2 = [i for i in processed_doc2 if i not in stopwords.words('english')]

print(processed_doc1,'\n', processed_doc2) 

为了更好地可视化处理后的文档,让我们创建它们的词云图。

In [9]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

print("Document 1:") 
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(' '.join(processed_doc1))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

print("Document 2:")
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(' '.join(processed_doc2))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

plt.show()

为了准备优化模型的文档,让我们存储每个词的出现频率。

In [10]:
freqency_D1 = {i: processed_doc1.count(i)/len(processed_doc1) for i in processed_doc1}
freqency_D2 = {i: processed_doc2.count(i)/len(processed_doc2) for i in processed_doc2}

D1 = set(processed_doc1)
D2 = set(processed_doc2) 
print(D1,D2)

现在我们得到词对词的距离矩阵。给定两个词的向量嵌入$\overline{x}_1$和$\overline{x}_2$,我们取它们之间的余弦距离,即$\overline{x}_1 . \overline{x}_2$/$|\overline{x}_1||\overline{x}_2|$。取消下面代码的注释可以看到两个文档中所有词对之间的排序距离。

In [11]:
import numpy as np
distance = {(i,j): spatial.distance.cosine(model[i],model[j]) for i in D1 for j in D2} 

# dict(sorted(distance.items(), key=lambda item: item[1]))

## III. 使用线性规划计算文本差异度

现在,我们将词移距离(WMD)建模为优化问题。
基本思想是从文档1中的词向文档2中的词发送*流量*,使得距离与流量的乘积最小化。
换句话说,我们在语义上更接近的词对之间发送更大的流量。

在下图中,我们可以看到最优流量对应于语义上最接近的词对。

|<img src="https://raw.githubusercontent.com/Gurobi/modeling-examples/master/text_dissimilarity/figure_obama2.png" width="500" align="center">| 
|:--:|
|用词移距离来衡量两份文档的相似度示意图。<b>图片来源: [Towards AI](https://towardsai.net/p/nlp/word-movers-distance-wmd-explained-an-effective-method-of-document-classification-89cb258401f4) </b>| 

<!-- 得分:距离的加权和。 -->

在优化术语中,这个模型被称为**运输模型**,其中流量对应于在不同位置之间运输商品(如仓库和零售点之间)。


在构建模型之前,我们先定义输入参数。

### 输入参数

$D_1, D_2$: 两个文档,每个文档代表一组词,

$p_w$: 词$w$在$D_1$中的频率,

$q_{w'}$: 词$w'$在$D_2$中的频率,

$d(w,w')$: 词$w$和$w'$的词向量嵌入之间的距离。

要构建模型,我们首先初始化Gurobi模型。

In [ ]:
%pip install gurobipy
import gurobipy as gp
from gurobipy import GRB

# 初始化模型
m = gp.Model("Text_similarity")

### 流量变量

以下是模型中的关键决策变量。

$f_{w,w'}$: 从$D_1$中的词$w$到$D_2$中的词$w'$的流量。



我们可以使用addVars函数为所有词对添加流量变量。我们允许流量变量在$0$和$1$之间,因此将下界(lb)设为$0$,上界(ub)设为$1$。

In [13]:
f = m.addVars(D1,D2,name="f",lb=0,ub=1) 

### 目标函数:最小化距离与流量的乘积总和


流量越大,这些词在语义上应该越接近。这是通过定义目标函数为距离和流量的乘积来实现的,用数学表达式为:

\begin{aligned}
\textrm{minimize} \ \sum_{w \in D1} \sum_{w' \in D2}&  d(w,w') f_{w,w'}
\end{aligned}



最小化这个目标函数自然会为距离较小的词对分配较高的流量值。我们可以在下面将这个目标函数添加到Gurobi模型中。

In [14]:
m.ModelSense = GRB.MINIMIZE
m.setObjective(sum(f[w,w_prime]*distance[w,w_prime] for w in D1 for w_prime in D2))#/sum(cost[w,w_prime] for w in D for w_prime in D_prime))
m.update()

### 约束条件


最后,我们定义约束条件来确保每个词在流量中的表示与其出现频率成比例。
我们通过确保从文档$D1$中每个词$w$流出的净流量等于它在$D1$中的出现频率来实现这一点。
同样,流入文档$D2$中每个词$w'$的净流量等于它在$D2$中的出现频率。
这两个约束条件可以用下面的两个方程表示。

\begin{aligned}
 \sum_{w' \in D2} f_{w,w'} &= p_w \quad  \forall \ w \in D1, \\
\ \sum_{w \in D1} f_{w,w'} &= q_{w'} \quad  \forall  \ w' \in D2
\end{aligned}

我们可以使用addConstrs函数将这些约束条件添加到Gurobi模型中。

In [ ]:
m.addConstrs(f.sum(w, '*') == freqency_D1[w] for w in D1)
m.addConstrs(f.sum('*', w_prime) == freqency_D2[w_prime] for w_prime in D2)

### 求解模型

我们已经添加了所有的决策变量、目标函数和约束条件,现在可以求解模型了!

In [16]:
m.optimize()  

模型已求解,流量解被存储在Pandas数据框中以便于可视化。

我们如何解释这个结果?多大的差异度才足以检测抄袭?

In [17]:
print("Dissimilarity score:",round((m.ObjVal),2),"\n")

solution = pd.DataFrame()
flow = {(i,j): f[i,j].X for i in D1 for j in D2 if f[i,j].X > 0} 
# flow = sorted(flow.items(), key=lambda item: item[1],reverse=True)
solution['word 1'] = [i for (i,j) in flow]
solution['word 2'] = [j for (i,j) in flow]
solution['flow'] = [flow[i,j] for (i,j) in flow]
solution['distance'] = [distance[i,j] for (i,j) in flow] 
# solution.sort_values(by='flow',ascending=False).reset_index(drop=True)
solution.sort_values(by='distance',ascending=True).reset_index(drop=True)

## IV. 检测抄袭

在本notebook的这一部分中,我们检查一段给定的文本是否是从一本书中重写(或抄袭)的。为此,我们将给定的文本与书中的每个句子进行比较,并输出差异度最小的句子。之后,人工可以对这是否确实是抄袭案例做出最终评估。

首先,将这本书(《福尔摩斯探案集》)作为文本文件读入。我们从[Project Gutenberg](https://www.gutenberg.org/cache/epub/1661/pg1661.txt)下载了这个文件。

In [18]:
import base64
import requests

master = "https://raw.githubusercontent.com/Gurobi/modeling-examples/master/text_dissimilarity/PG1661_raw.txt"
content = requests.get(master)
content = content.text
content = content.replace('\n',' ')
content = content.replace('_',' ')
content = content.replace('\r','') 
sentences = list(map(str.strip, content.split(".")))[19:]

对书中的所有句子进行预处理。

In [ ]:
def pre_processing(document):
    # 去掉专有名词
    tagged_doc = nltk.tag.pos_tag(document.split())
    edited_sentence = [word for word,tag in tagged_doc]  
    edited_sentence = [word for word,tag in tagged_doc if tag not in ['NNP','NNPS']]  

    # 删除标点符号
    tokenizer = RegexpTokenizer(r'\w+') 
    processed_doc = tokenizer.tokenize(' '.join(edited_sentence)) 

    # 删除停止词
    processed_doc = [i for i in processed_doc if i not in stopwords.words('english')]  
    
    return processed_doc

processed_sentences = [] # 所有句子列表
for s in sentences: 
    processed_sentences.append(pre_processing(s))


我们可以将WMD优化模型写在一个函数中,该函数输入两个文档,输出它们的差异度分数。

In [ ]:
def score_dissimilarity(D1, D2):

    D1 = set(D1)
    D2 = set(D2) 
    D2 = D2 - set([i for i in D2 if i not in model]) # 如果书中的某些单词不在word2vec数据中
    D1 = D1 - set([i for i in D1 if i not in model])
            
    freqency_D1 = {i: list(D1).count(i)/len(D1) for i in D1}
    freqency_D2 = {i: list(D2).count(i)/len(D2) for i in D2}
    
    if len(D2) < 5: # 如果句子太小，我们设置高不相似度，有效地忽略它
        return 1
        
    m = gp.Model("Text_similarity")
    distance = {(i,j): spatial.distance.cosine(model[i],model[j]) for i in D1 for j in D2} 

    # 变量。调整这里的边界
    f = m.addVars(D1,D2,name="f",lb=0,ub=1) 

    # 最小化
    m.ModelSense = GRB.MINIMIZE
    m.setObjective(sum(f[w,w_prime]*distance[w,w_prime] for w in D1 for w_prime in D2))

    # 添加约束
    m.addConstrs(f.sum(w, '*') ==  freqency_D1[w] for w in D1)
    m.addConstrs(f.sum('*', w_prime) == freqency_D2[w_prime] for w_prime in D2) 
    
    m.setParam('OutputFlag', 0)
    m.optimize()  
    
    return m.ObjVal

现在,我们选择一段从书中重写的段落。下面的代码有一些示例,但你也可以从[这本书](https://www.gutenberg.org/cache/epub/1661/pg1661.txt)中选择任何句子并创建你自己的抄袭版本。

In [21]:
sample_sentence = 'With a gaze that shifted back and forth between us, the dimunitive figure appeared to be a mixture of apprehension and anticipation, uncertain if he was on the cusp of a fortune or a disaster.'
# sample_sentence = 'Without much conversation, yet with a friendly gesture, he gestured towards an armchair for me to sit in, offered me a box of cigars, and pointed to a liquor cabinet and a carbonated water dispenser in the corner.' 

print(sample_sentence)

最后,我们遍历书中的所有句子,并找出差异度分数。下面的代码只在差异度降低时打印输出。结果是差异度最小的句子。

In [22]:
obj_best = 1
print("#\t Dissimilarity\t Sentence")
for i in range(len(processed_sentences)):  
    obj = score_dissimilarity(pre_processing(sample_sentence),processed_sentences[i])  
    if obj < obj_best: 
        print(i,"\t",round((obj),2),"\t",sentences[i])
        obj_best, sentence_best = obj, sentences[i]

print("\nThe closest sentence with a %f dissimilarity is:\n\n"%obj_best,sentence_best) 

太好了!程序是否正确识别了段落?试试其他段落及其重写版本吧!

Copyright © 2023 Gurobi Optimization, LLC